In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

# use this colab tool to upload files from local storage into colab notebook


# Dataset used here is the combination iof NSL-KDD train and test text files
# seting feature columns using numpy array


labels = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'attack_type', 'difficulty_level']# subclass - > attack_type



train_ds = pd.read_csv("/content/drive/My Drive/RM2/KDDTotal.txt", 
                       dtype= str, 
                       encoding= "utf-8")



train_ds.columns = labels 
combined_data = train_ds.drop('difficulty_level', 1)
for i in range(0,len(combined_data)):
 
  if combined_data[ 'attack_type'][i] != "normal":
    combined_data['attack_type'][i] = "anomaly"


In [0]:
len(combined_data)

148516

In [0]:
cat_col = labels[1:4]

In [0]:
combined_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.00,0.00,0.00,0.00,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.00,1.00,0.00,0.00,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
2,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.20,0.20,0.00,0.00,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.00,0.00,0.00,0.00,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.00,0.00,1.00,1.00,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148511,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal
148512,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.00,0.00,0.00,0.00,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal
148513,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.00,0.00,0.00,0.00,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,anomaly
148514,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.00,0.00,0.00,0.00,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [0]:
from sklearn.preprocessing import LabelEncoder,normalize
le = LabelEncoder()

vector = combined_data['attack_type']

print("Attack Vectors:", set(list(vector))) # use print to make it print on single line 
combined_data['attack_type'] = le.fit_transform(vector)
combined_data['protocol_type'] = le.fit_transform(combined_data['protocol_type'])
combined_data['service'] = le.fit_transform(combined_data['service'])
combined_data['flag'] = le.fit_transform(combined_data['flag'])

print('\nDescribing attack_type: ')
print("min", vector.min())
print("max", vector.max())
print("mode",vector.mode())
print("mode", len(np.where(vector.values==16)[0])/len(vector),"%")


Attack Vectors: {'anomaly', 'normal'}

Describing attack_type: 
min anomaly
max normal
mode 0    normal
dtype: object
mode 0.0 %
looks like 16 is 'normal' 


In [0]:
combined_data.info

<bound method DataFrame.info of        duration  protocol_type  ...  dst_host_srv_rerror_rate  attack_type
0             0              2  ...                      0.00            1
1             0              1  ...                      0.00            0
2             0              1  ...                      0.01            1
3             0              1  ...                      0.00            1
4             0              1  ...                      1.00            0
...         ...            ...  ...                       ...          ...
148511        0              1  ...                      0.00            1
148512        0              1  ...                      0.00            1
148513        0              1  ...                      0.07            0
148514        0              2  ...                      0.00            1
148515        0              1  ...                      1.00            0

[148516 rows x 42 columns]>

In [0]:
data_x = combined_data.drop('attack_type', axis=1)
data_y = combined_data.loc[:,['attack_type']]
# del combined_data # free mem

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.2, random_state=42) # TODO

X_train = pd.DataFrame(normalize(X_train))
X_test = pd.DataFrame(normalize(X_test)) # this is worng (cheating), i should have used a sk-learn obj. 

In [0]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.000000,0.003430,0.168086,0.017152,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.449374,0.048025,0.003396,0.003430,0.000000,0.000000,0.000377,0.000240,0.0,0.874735,0.044594,0.000172,0.000206,0.000000,0.0,3.430335e-03,3.430335e-03,0.000000,0.000000
1,0.000000,0.003550,0.173927,0.017748,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.383350,0.053243,0.003550,0.003550,0.000000,0.000000,0.000497,0.000213,0.0,0.905132,0.017748,0.000071,0.000213,0.000000,0.0,3.549537e-03,3.549537e-03,0.000000,0.000000
2,0.000000,0.001249,0.029980,0.011243,0.301051,0.834447,0.0,0.0,0.0,0.0000,0.0,0.001249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067455,0.067455,0.000000,0.000000,0.000000,0.000000,0.001249,0.000000,0.0,0.318539,0.318539,0.001249,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000
3,0.000000,0.002674,0.061508,0.013371,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727395,0.040114,0.002674,0.002674,0.000000,0.000000,0.000160,0.000160,0.0,0.681933,0.016045,0.000053,0.000187,0.000000,0.0,2.674246e-03,2.674246e-03,0.000000,0.000000
4,0.000000,0.007068,0.173169,0.031807,0.003534,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395815,0.007068,0.000000,0.000000,0.000000,0.000000,0.000071,0.003499,0.0,0.901186,0.007068,0.000035,0.003004,0.003534,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74253,0.000000,0.003451,0.169121,0.017257,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.434884,0.062126,0.003451,0.003451,0.000000,0.000000,0.000483,0.000242,0.0,0.880122,0.058675,0.000242,0.000242,0.000000,0.0,3.451457e-03,3.451457e-03,0.000000,0.000000
74254,0.000000,0.003382,0.223227,0.016911,0.000000,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.453218,0.016911,0.003382,0.003382,0.000000,0.000000,0.000135,0.000237,0.0,0.862466,0.016911,0.000068,0.000271,0.000000,0.0,3.382221e-03,3.382221e-03,0.000000,0.000000
74255,0.078303,0.001450,0.027551,0.013051,0.214609,0.932388,0.0,0.0,0.0,0.0029,0.0,0.001450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001450,0.001450,0.000000,0.000000,0.000000,0.000000,0.001450,0.000000,0.0,0.278411,0.001450,0.000015,0.000058,0.000015,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000
74256,0.498815,0.000555,0.006103,0.001110,0.866683,0.000000,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000555,0.000555,0.000000,0.000000,0.000555,0.000555,0.000555,0.000000,0.0,0.001110,0.001110,0.000555,0.000000,0.000277,0.0,0.000000e+00,0.000000e+00,0.000555,0.000555


In [0]:


from sklearn.neighbors import KNeighborsClassifier as KNC

# SVC classifier with various parameters is fitted with input/output pielines

def clf(n, leaf_size, metric):
  
  clf = KNC(n_neighbors=n, weights='uniform', algorithm='auto', leaf_size=leaf_size , p=2, 
        metric= metric , metric_params=None, n_jobs=None)  'minkowski'

  clf.fit(X_train, y_train)

  # predicting accuracy of model with testing data

  predictions = clf.predict(X_test)

  # scoring overall trained model on new data

  from sklearn import metrics
  score = metrics.accuracy_score(y_test, predictions)
  return print("score of KNN is:", score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


score of KNN is: 0.9870051171559386
